# 1.Load Data lên Bigquery

In [1]:
import pandas as pd 
import pyarrow
from google.cloud import bigquery
from google.oauth2 import service_account


In [ ]:
df = pd.read_excel(r"D:\DA\LEGO timesheet\Data\Sample Data copy.xlsx")

In [ ]:
df.shape

In [ ]:
# Các cột có null là Resource group, Country, Time Approver, Approved By, Date Time Approved
# Tuy nhiên Date Time Last Amended có nên chuyển hết thành null ko?
df.info()

In [ ]:
df[df.duplicated(keep=False)]

In [ ]:
df_clean = df.drop_duplicates(keep='first')
df_clean.shape
#103961 rows, 3703 dup rows --> df_clean = 101940

In [ ]:
df_clean.duplicated().sum() ## ko còn dup rows --> clean'
## sửa định dạng snake_case
df_clean.columns = (
    df_clean.columns.str.strip().str.lower().str.replace(' ', '_')
)
## test thử xem có bao nhiêu dòng miss 'approved by'
test = df_clean[df_clean['approved_by'] == 'not found']
test.shape

In [ ]:
## load sheet 2
df_2 = pd.read_excel(r"D:\DA\LEGO timesheet\Data\Sample Data copy.xlsx", sheet_name='Sheet2')
df_2.columns = (
    df_2.columns.str.strip().str.lower().str.replace(' ', '_')
)
## bq ko chấp nhận rate/hr --> rate_per_hour
df_2 = df_2.rename(columns= {'rate/hr' : 'rate_per_hour'})
df_2.head()
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   resource_group  6 non-null      object
 1   rate_per_hour   6 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 228.0+ bytes


In [12]:
##load sheet3
df_3 = pd.read_excel(r"D:\DA\LEGO timesheet\Data\Sample Data copy.xlsx", sheet_name='Sheet3')
df_3.columns = (
    df_3.columns.str.strip().str.lower().str.replace(' ', '_')
)
df_3 = df_3.rename(columns= {'contractual_hrs/week' : 'contractual_hours_per_week'})
df_3.head()

,country,contractual_hours_per_week
0,Germany,37.5
1,Mexico,40.0
2,PRC,40.0


In [ ]:
## đẩy lên bigquery
project_id = "jda-k1"
dataset_id = "practice_data_pipeline"
table_id = "dim_country"
key_path = r"D:\DA\LEGO timesheet\Key Path\jda-k1-2afad6d8f47e.json"

credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials=credentials, project=project_id) ##tạo client

table_ref = f"{project_id}.{dataset_id}.{table_id}"

job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    autodetect=True,  
)

job = client.load_table_from_dataframe(df_3, table_ref, job_config=job_config) ##cấu hình ghi truncate hoặc append
job.result()  

print(f"Load thành công {job.output_rows} dòng vào bảng {table_ref}")



c:\Users\Admin\AppData\Local\Programs\Python\Python314\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Load thành công 3 vào bảng jda-k1.practice_data_pipeline.dim_country
